#Set-Up

In [ ]:
#Copy-and-paste the code below to use as "set-up" when your optimization model uses Pyomo and Coin-OR solvers.
#for reference, see https://jckantor.github.io/ND-Pyomo-Cookbook/notebooks/01.02-Running-Pyomo-on-Google-Colab.html#installing-pyomo-and-solvers

%%capture
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

from pyomo.environ import *

#Review: A Simple Optimization Problem
minimize f(x) = 4x^4 - 3x^3 - 9x + 20

In [ ]:
#initialize a "Concrete Model"
model = ConcreteModel()

#initialize DVs
model.x = Var(domain = Reals) #You can google "pyomo domain sets" to see several domain examples

#define the objective
model.Objective = Objective(expr = 4*model.x**4 - 3*model.x**3 - 9*model.x + 20, sense = minimize)

#(Optional) You can use model.pprint() to see what you've done so far
model.pprint()

1 Var Declarations
    x : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  None :  None : False :  True :  Reals

1 Objective Declarations
    Objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : 4*x**4 - 3*x**3 - 9*x + 20

2 Declarations: x Objective


In [ ]:
#solve model
opt = SolverFactory('ipopt')

results = opt.solve(model, tee = True) #setting tee = False hides the diagnostic outputs

Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. See http://

In [ ]:
#print relevant values
print("x* = ", model.x()) #alternatively you can use value(model.x)
print("obj* = ", model.Objective()) #alternatively you can use value(model.Objective)

x* =  1.0616077811644107
obj* =  11.936814677331192


##Repeating with Objective as a Function


In [ ]:
#define the objective function
def obj_function(x):
  output = 4*x**4 - 3*x**3 - 9*x + 20
  return output

In [ ]:
#initialize a "Concrete Model"
model2 = ConcreteModel()

#initialize DVs
model2.x = Var(domain = Reals) #You can google "pyomo domain sets" to see several domain examples

#define the objective
model2.Objective = Objective(expr = obj_function(model2.x), sense = minimize)

#(Optional) You can use model.pprint() to see what you've done so far
model2.pprint()

1 Var Declarations
    x : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  None :  None : False :  True :  Reals

1 Objective Declarations
    Objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : 4*x**4 - 3*x**3 - 9*x + 20

2 Declarations: x Objective


In [ ]:
#solve model & print relevant values
opt = SolverFactory('ipopt')

results = opt.solve(model2, tee = False) #setting tee = False hides the diagnostic outputs

#print relevant values
print("x* = ", model2.x()) #alternatively you can use value(model.x)
print("obj* = ", model2.Objective()) #alternatively you can use value(model.Objective)

x* =  1.0616077811644107
obj* =  11.936814677331192


#Demo: Optimizing Existing Models (Adbudget)
We created the below model in the previous module. Use ipopt in Pyomo to find the optimal price.

In [ ]:
def ad_profit(unit_cost = 25,
              sales_price = 40,
              seasonality_factor = [.9, 1.1, .8, 1.2],
              salesforce_cost = [8000, 8000, 9000, 9000],
              overhead_rate = .15,
              k = 35,
              b = 3000,
              ad_spend = [10000, 10000, 10000, 10000],
              num_quarters = 4):
  #Gross profit calculations
  sales_quantity = [k*seasonality_factor[i]*((b + ad_spend[i])**.5) for i in range(num_quarters)]
  cogs = [unit_cost*sales_quantity[i] for i in range(num_quarters)]
  sales_rev = [sales_price*sales_quantity[i] for i in range(num_quarters)]
  gross_profit = [sales_rev[i] - cogs[i] for i in range(num_quarters)]
  total_gross_profit = sum(gross_profit)
  #operating expense calculation
  overhead = [overhead_rate*sales_rev[i] for i in range(num_quarters)]
  operating_expenses = [overhead[i] + salesforce_cost[i] + ad_spend[i] for i in range(num_quarters)]
  total_operating_expenses = sum(operating_expenses)
  #operating profit
  operating_profit = total_gross_profit - total_operating_expenses
  return operating_profit

In [ ]:
#initialize a concrete model
ad_model = ConcreteModel()

#initialize DVs
ad_model.ad_spend_dv = Var(range(4), domain=NonNegativeReals)

#define the objective
ad_model.obj = Objective(expr = ad_profit(ad_spend = ad_model.ad_spend_dv), sense = maximize)

#model pprint
ad_model.pprint()

1 Var Declarations
    ad_spend_dv : Size=4, Index={0, 1, 2, 3}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True : NonNegativeReals
          1 :     0 :  None :  None : False :  True : NonNegativeReals
          2 :     0 :  None :  None : False :  True : NonNegativeReals
          3 :     0 :  None :  None : False :  True : NonNegativeReals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : 40*(31.5*(3000 + ad_spend_dv[0])**0.5) - 25*(31.5*(3000 + ad_spend_dv[0])**0.5) + 40*(38.5*(3000 + ad_spend_dv[1])**0.5) - 25*(38.5*(3000 + ad_spend_dv[1])**0.5) + 40*(28.0*(3000 + ad_spend_dv[2])**0.5) - 25*(28.0*(3000 + ad_spend_dv[2])**0.5) + 40*(42.0*(3000 + ad_spend_dv[3])**0.5) - 25*(42.0*(3000 + ad_spend_dv[3])**0.5) - (0.15*(40*(31.5*(3000 + ad_spend_dv[0])**0.5)) + 8000 + ad_spend_dv[0] + 0.15*(40*(38.5*(3000 + ad_spend_dv[1])**0

In [ ]:
#solve model
opt = SolverFactory('ipopt')

results = opt.solve(ad_model, tee = True) #setting tee = False hides the diagnostic outputs

Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. See http://

In [ ]:
#print relevant values
print(f"Optimal operating profit is {ad_model.obj()}")
print(f"Optimal ad_spend in Q1 is {ad_model.ad_spend_dv[0]()}")
print(f"Optimal ad_spend in Q2 is {ad_model.ad_spend_dv[1]()}")
print(f"Optimal ad_spend in Q3 is {ad_model.ad_spend_dv[2]()}")
print(f"Optimal ad_spend in Q4 is {ad_model.ad_spend_dv[3]()}")


Optimal operating profit is 79705.62500000003
Optimal ad_spend in Q1 is 17093.06250000487
Optimal ad_spend in Q2 is 27015.562500004056
Optimal ad_spend in Q3 is 12876.00000000539
Optimal ad_spend in Q4 is 32721.00000000369
